In [1]:
import os
import re
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import scorpyo as sp

from null_importance import get_null_importance


pd.set_option('max_rows', 500, 'max_columns', 200)

In [2]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')
path_results_jupyter  = os.path.join(path_results, 'jupyter')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')


path_new_train = os.path.join(path_new_data, 'train_lightgbm_20221014.csv')
path_new_test  = os.path.join(path_new_data, 'test_lightgbm_20221014.csv')

## results
path_output_report = os.path.join(path_results, '01_原始数据探察_20221014.xlsx')

y_label = "is_risk"

In [3]:
df_row_train = sp.read_data(path_train)
df_row_val  = sp.read_data(path_test)

df_row_train['url_sit'] = df_row_train['url'].map(lambda x: x.split('/')[0])
df_row_train['url_page'] = df_row_train['url'].map(lambda x: x.split('/')[1])

df_row_val['url_sit'] = df_row_val['url'].map(lambda x: x.split('/')[0])
df_row_val['url_page'] = df_row_val['url'].map(lambda x: x.split('/')[1])



df_train_info = df_row_train.copy().sort_values(by='op_datetime')

In [4]:
df_row_train.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk,url_sit,url_page
0,0,guojianping9672,rd,GVhZtW4i1,rqRxAjAL1RYC,firefox_78,firefox,win,win10,2022-01-18 19:10:41,内网,200,成都,2umVQwhiiwNJ,xxx.com/mail,2022-01,0,xxx.com,mail
1,1,yangtao1740,sales,l3MuTMPoQ,iKPTa3su50y7,chrome_93,chrome,win,win11,2022-04-01 17:04:00,内网,200,深圳,RwHe8Q1R7AlB,business.xxx.com/,2022-04,0,business.xxx.com,
2,2,wangying9098,rd,4uHWcskWv,1baNbqxMWcCu,ie_11,ie,win,win10,2022-03-01 15:53:49,内网,200,成都,dwS3cdn15GK4,wpsdoc.xxx.com/kdocs,2022-03,0,wpsdoc.xxx.com,kdocs
3,3,liguixiang3860,rd,mQh3NwtY7,C04Llg4lKl4C,edge_93,edge,win,win10,2022-02-07 19:46:25,内网,200,北京,nHrKgKdJ1Mzt,xxx.com/github,2022-02,0,xxx.com,github
4,4,guanyu9205,sales,C2QtgDKAZ,kSscjiRSz1aD,edge_93,edge,win,win10,2022-04-12 10:05:19,内网,200,成都,RwHe8Q1R7AlB,business.xxx.com/,2022-04,0,business.xxx.com,


In [5]:
sp.excel_detect(df_row_train, sheet_name='02.训练集-数据描述性统计')

sheet02.训练集-数据描述性统计在01_数据探查.xlsx中已经存在，我们将对原文件进行覆盖


,feat_name_row,type,size,missing,unique,zero_ratio,negative_ratio,top1_all_value,top1_all_ratio,mean_or_top1,std_or_top2,min_or_top3,1%_or_top4,10%_or_top5,50%_or_bottom5,75%_or_bottom4,90%_or_bottom3,99%_or_bottom2,max_or_bottom1
0,id,int64,47660,0.00,47660,0.000,0.0,0,0.000,23829.5,13758.401,0.0,476.59,4765.9,23829.5,35744.25,42893.1,47182.41,47659.0
1,user_name,object,47660,0.08,187,0.000,NaN,xuxiuying8050,0.007,xuxiuying8050:0.65%,hongchang3029:0.63%,tanliu3173:0.62%,liuhong6350:0.62%,lufan2545:0.62%,zhouxiumei4433:0.38%,chenjian4844:0.37%,wanggang1192:0.36%,ranxiuzhen6780:0.33%,xujie9775:0.30%
2,department,object,47660,0.08,5,0.000,NaN,rd,0.654,rd:65.36%,sales:17.26%,other:4.07%,accounting:3.56%,hr:1.75%,rd:65.36%,sales:17.26%,other:4.07%,accounting:3.56%,hr:1.75%
3,ip_transform,object,47660,0.00,2105,0.000,NaN,w2CfuqTz3,0.007,w2CfuqTz3:0.68%,u9diCFdYZ:0.66%,pPgzIf3S4:0.65%,7YnPN3fqd:0.65%,DhTMwbtS5:0.64%,948U9MQcB:0.00%,h75YAkAAL:0.00%,m7512MutA:0.00%,ADL8GwW32:0.00%,g3dWezpzT:0.00%
4,device_num_transform,object,47660,0.00,844,0.000,NaN,O54DfqjlCrhL,0.007,O54DfqjlCrhL:0.70%,kUa61ygA6gI3:0.68%,Rfv57YyO3vny:0.67%,5DmlITfRNR36:0.66%,TzmgdvYq3Kx0:0.66%,aUECyyFo55Zy:0.00%,cREgOG9x3d9X:0.00%,NGfeE42d1yHY:0.00%,T4hueKNccs7X:0.00%,A0TLDctT8OUR:0.00%
5,browser_version,object,47660,0.00,8,0.000,NaN,edge_93,0.372,edge_93:37.16%,chrome_90:33.38%,safari_13:9.48%,chrome_77:5.32%,firefox_78:4.82%,chrome_77:5.32%,firefox_78:4.82%,chrome_93:4.44%,ie_11:3.37%,ie_9:2.02%
6,browser,object,47660,0.00,5,0.000,NaN,chrome,0.431,chrome:43.15%,edge:37.16%,safari:9.48%,ie:5.39%,firefox:4.82%,chrome:43.15%,edge:37.16%,safari:9.48%,ie:5.39%,firefox:4.82%
7,os_type,object,47660,0.00,2,0.000,NaN,win,0.905,win:90.52%,macos:9.48%,None,None,None,None,None,None,win:90.52%,macos:9.48%
8,os_version,object,47660,0.00,4,0.000,NaN,win10,0.758,win10:75.84%,win7:10.81%,macos_big_sur_11:9.48%,win11:3.87%,None,None,win10:75.84%,win7:10.81%,macos_big_sur_11:9.48%,win11:3.87%
9,op_datetime,object,47660,0.00,47343,0.000,NaN,2022-04-02 17:02:24,0.000,2022-04-02 17:02:24:0.01%,2022-02-16 11:36:12:0.01%,2022-04-28 09:33:53:0.01%,2022-03-04 17:13:36:0.01%,2022-03-23 14:47:36:0.00%,2022-04-29 09:58:59:0.00%,2022-04-20 19:02:48:0.00%,2022-01-19 15:11:24:0.00%,2022-01-19 10:53:46:0.00%,2022-04-08 19:30:10:0.00%


## 定义一次/一天/一月
1. op\_times\_groups 一次
2. op\_days 一天
3. op\_month 一月

In [6]:
df = pd.concat([df_row_train, df_row_val]).reset_index(drop=True)
df = df.sort_values(by='op_datetime')
# 认证日期时间
df['op_datetime'] = pd.to_datetime(df['op_datetime'])


# 将数据分为每一天
df['op_days'] = df['op_datetime'].map(lambda x: x.strftime('%Y-%m-%d'))

# 按客户进行统计这次认证和上次认证的时间差
df['op_second'] = df['op_datetime']
df['op_second1'] = df.groupby('device_num_transform')['op_second'].shift(1)

df['op_diff_second1'] = (df['op_second'] - df['op_second1']).map(lambda x: x.total_seconds())
df['op_diff_second1_tmp'] = df['op_diff_second1']>10
# 客户第几次登录
df['op_times_groups'] = df.groupby('device_num_transform')['op_diff_second1_tmp'].apply(lambda x: x.cumsum())

df = df.drop(columns=['op_second','op_second1','op_diff_second1_tmp'])


## 网络嗅探相应特征生成

1.考虑特征范围为，客户+时间+ 按时间cumsum/cumunique

In [ ]:
time_feats = ['op_times_groups', 'op_days', 'op_month']

cate_feats = ['department', 'ip_transform', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url_page','url_sit']

# 客户+时间+按时间cumsum/cumunique
# 累计量统计
df['helper'] = 1
# 是否为异常状态码
df['http_status_code_helper'] = df['http_status_code'].map(lambda x: 1 if x in [400, 500, 502, 404] else 0) 
# 给样本编号
df['sampler_index_helper'] = df['helper'].cumsum()

for i in time_feats:
    # cumsum
    df['device_num_transform_{}_cumsum'.format(i)] = df.groupby(['device_num_transform', i])['helper'].cumsum()
    # cumunique
    for j in cate_feats:
        index_set = set(df.groupby(['device_num_transform', i, j], as_index=False).first()['sampler_index_helper'].values)
        df['tmp_helper'] = df['sampler_index_helper'].map(lambda x: 1 if x in index_set else 0)
        df['device_num_transform_{}_{}_cumunique'.format(i, j)] = df.groupby(['device_num_transform', i])['tmp_helper'].cumsum()

        
remove_cols = [x for x in df.columns if x[-6:]=='helper']

df = df.drop(columns=remove_cols)

## 暴力破解
1. 单个mac/ip/用户认证失败次数


In [8]:
time_feats = ['op_times_groups', 'op_days', 'op_month']

object_feats = ['ip_transform', 'user_name', 'device_num_transform']

# 累计量统计
df['helper'] = 1
# 是否为异常状态码
df['http_status_code_helper'] = df['http_status_code'].map(lambda x: 1 if x in [400, 500, 502, 404] else 0) 
# 给样本编号
df['sampler_index_helper'] = df['helper'].cumsum()

for i in time_feats:
    # 系统层面异常次数
    df['http_status_code_{}_cumsum'.format(i)] = df.groupby([i])['http_status_code_helper'].cumsum()
    for j in object_feats:
        # object层面异常次数
        df['{}_{}_http_status_code_error_code_cumsum'.format(j,i)] = df.groupby([i,j])['http_status_code_helper'].cumsum()
        
        # object层面异常个数
        index_set = set(df[df['http_status_code'].isin([400, 500, 502, 404])].groupby([i, j,  'http_status_code'], as_index=False).first()['sampler_index_helper'].values)
        df['tmp_helper'] = df['sampler_index_helper'].map(lambda x: 1 if x in index_set else 0)
        df['{}_{}_http_status_code_error_code_cumunique'.format(j,i)] = df.groupby([i,j])['tmp_helper'].cumsum()

remove_cols = [x for x in df.columns if x[-6:]=='helper']

df = df.drop(columns=remove_cols)

## 账号异常行为
1. 账号+ip的登录次数
2. 账号的异常登录时间
3. 账号+ip的异常登录时间

In [9]:
# 几点钟
df['hour'] = df['op_datetime'].dt.hour
df['hour_sin'] = np.sin(df['hour']/24*2*np.pi)
df['hour_cos'] = np.cos(df['hour']/24*2*np.pi)

# 周几
df['dayofweek'] = df['op_datetime'].dt.dayofweek
df['dayofweek_sin'] = np.sin(df['dayofweek']/7*2*np.pi)
df['dayofweek_cos'] = np.cos(df['dayofweek']/7*2*np.pi)

# 一个月的第几天
df['day'] = df['op_datetime'].dt.day
df['day_sin'] = np.sin(df['day']/31*2*np.pi)
df['day_cos'] = np.cos(df['day']/31*2*np.pi)

In [10]:
# time_feats = ['op_times_groups', 'op_days', 'op_month']

# object_feats = ['ip_transform', 'user_name', 'device_num_transform']

# cate_feats = ['department', 'browser_version', 'browser', 'os_type','os_version',
#               'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url_page','url_sit']

# for i in time_feats:
#     for j in object_feats:
#         for z in range(3, 10, 2):
#             tmp = df.groupby([i,j],as_index=False)['op_diff_second1'].rolling(z)

#             df['{}_{}_{}_op_diff_second1_mean'.format(i,j,z)] = tmp.mean()['op_diff_second1']
#             df['{}_{}_{}_op_diff_second1_sum'.format(i,j,z)] = tmp.sum()['op_diff_second1']
#             df['{}_{}_{}_op_diff_second1_std'.format(i,j,z)] = tmp.std()['op_diff_second1']
#             df['{}_{}_{}_op_diff_second1_var'.format(i,j,z)] = tmp.var()['op_diff_second1']
#         for k in cate_feats:
#             for z in range(3, 10, 2):
#                 tmp_t = df.groupby([i,j,k],as_index=False)['op_diff_second1'].rolling(z)
#                 df['{}_{}_{}_{}_op_diff_second1_mean'.format(i,j,k, z)] = tmp_t.mean()['op_diff_second1']
#                 df['{}_{}_{}_{}_op_diff_second1_sum'.format(i,j,k, z)] = tmp_t.sum()['op_diff_second1']
#                 df['{}_{}_{}_{}_op_diff_second1_std'.format(i,j,k, z)] = tmp_t.std()['op_diff_second1']
#                 df['{}_{}_{}_{}_op_diff_second1_var'.format(i,j,k, z)] = tmp_t.var()['op_diff_second1']

            

In [11]:
# time_feats = ['op_times_groups', 'op_days', 'op_month']

# object_feats = ['ip_transform', 'user_name', 'device_num_transform']

# cate_feats = ['department', 'browser_version', 'browser', 'os_type','os_version',
#               'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url_page','url_sit']

# for i in time_feats:
#     for j in object_feats:
#         for z in range(3, 10, 2):
#             df['{}_{}_{}_op_diff_second1_mean'.format(i,j,z)] = df.groupby([i,j],as_index=False)['op_diff_second1'].rolling(z).mean()['op_diff_second1']
#             df['{}_{}_{}_op_diff_second1_sum'.format(i,j,z)] = df.groupby([i,j],as_index=False)['op_diff_second1'].rolling(z).sum()['op_diff_second1']
#             df['{}_{}_{}_op_diff_second1_std'.format(i,j,z)] = df.groupby([i,j],as_index=False)['op_diff_second1'].rolling(z).std()['op_diff_second1']
#             df['{}_{}_{}_op_diff_second1_var'.format(i,j,z)] = df.groupby([i,j],as_index=False)['op_diff_second1'].rolling(z).var()['op_diff_second1']
# #         for k in cate_feats:
# #             for z in range(3, 10, 2):
# #                 df['{}_{}_{}_{}_op_diff_second1_mean'.format(i,j,k, z)] = df.groupby([i,j,k],as_index=False)['op_diff_second1'].rolling(z).mean()['op_diff_second1']
# #                 df['{}_{}_{}_{}_op_diff_second1_sum'.format(i,j,k, z)] = df.groupby([i,j,k],as_index=False)['op_diff_second1'].rolling(z).sum()['op_diff_second1']
# #                 df['{}_{}_{}_{}_op_diff_second1_std'.format(i,j,k, z)] = df.groupby([i,j,k],as_index=False)['op_diff_second1'].rolling(z).std()['op_diff_second1']
# #                 df['{}_{}_{}_{}_op_diff_second1_var'.format(i,j,k, z)] = df.groupby([i,j,k],as_index=False)['op_diff_second1'].rolling(z).var()['op_diff_second1']

            

In [12]:
# cat_f = ['user_name','department','ip_transform','device_num_transform','browser_version','browser',
#  'os_type','os_version','ip_type','op_city','log_system_transform','url','url_sit','url_page']


# # 删除时间及类别型变量过多的特征
# remove_col = ['op_datetime', 'op_month', 'user_name', 'ip_transform', 'device_num_transform', 'op_days', 'ts', 'ts1', 'ts2','ts3']


# for f in cat_f:
#     df[f+'_ts_diff_mean'] = df.groupby([f])['ts_diff1'].transform('mean')
#     df[f+'_ts_diff_std'] = df.groupby([f])['ts_diff1'].transform('std')
#     df[f+'_ts_diff2_mean'] = df.groupby([f])['ts_diff2'].transform('mean')
#     df[f+'_ts_diff2_std'] = df.groupby([f])['ts_diff2'].transform('std')
#     df[f+'_ts_diff3_mean'] = df.groupby([f])['ts_diff3'].transform('mean')
#     df[f+'_ts_diff3_std'] = df.groupby([f])['ts_diff3'].transform('std')

#     df['device_num_transform_op_days_' + f+'_ts_diff_mean'] = df.groupby(['device_num_transform','op_days',f])['ts_diff1'].transform('mean')
#     df['device_num_transform_op_days_' + f+'_ts_diff_std'] = df.groupby(['device_num_transform','op_days',f])['ts_diff1'].transform('std')
#     df['device_num_transform_op_days_' + f+'_ts_diff2_mean'] = df.groupby(['device_num_transform','op_days',f])['ts_diff2'].transform('mean')
#     df['device_num_transform_op_days_' + f+'_ts_diff2_std'] = df.groupby(['device_num_transform','op_days',f])['ts_diff2'].transform('std')
#     df['device_num_transform_op_days_' + f+'_ts_diff3_mean'] = df.groupby(['device_num_transform','op_days',f])['ts_diff3'].transform('mean')
#     df['device_num_transform_op_days_' + f+'_ts_diff3_std'] = df.groupby(['device_num_transform','op_days',f])['ts_diff3'].transform('std')
    
#     df['device_num_transform_op_month_' + f+'_ts_diff_mean'] = df.groupby(['device_num_transform','op_month',f])['ts_diff1'].transform('mean')
#     df['device_num_transform_op_month_' + f+'_ts_diff_std'] = df.groupby(['device_num_transform','op_month',f])['ts_diff1'].transform('std')
#     df['device_num_transform_op_month_' + f+'_ts_diff2_mean'] = df.groupby(['device_num_transform','op_month',f])['ts_diff2'].transform('mean')
#     df['device_num_transform_op_month_' + f+'_ts_diff2_std'] = df.groupby(['device_num_transform','op_month',f])['ts_diff2'].transform('std')
#     df['device_num_transform_op_month_' + f+'_ts_diff3_mean'] = df.groupby(['device_num_transform','op_month',f])['ts_diff3'].transform('mean')
#     df['device_num_transform_op_month_' + f+'_ts_diff3_std'] = df.groupby(['device_num_transform','op_month',f])['ts_diff3'].transform('std')



## 账号下载量异常
1. 平均object的下载次数
2. 平均查询时间

In [13]:
# 累计量统计
df['helper'] = 1

time_feats = ['op_times_groups', 'op_days', 'op_month']

object_feats = ['ip_transform', 'user_name', 'device_num_transform']

for i in time_feats:
    for j in object_feats:
        for f in df['url_page'].unique():
            df['{}_{}_{}_cumsum'.format(j, i, f)] = df[df['url_page']==f].groupby([i,j])['helper'].cumsum()
            for z in df['http_status_code'].unique():
                df['{}_{}_{}_{}_cumsum'.format(j, i, f, z)] = df[(df['url_page']==f)&(df['http_status_code']==z)].groupby([i,j])['helper'].cumsum()
        
remove_cols = [x for x in df.columns if x[-6:]=='helper']

df = df.drop(columns=remove_cols)

    

## 特征筛选

In [14]:
# 删除时间及类别型变量过多的特征
remove_col = ['op_datetime', 'op_month', 'user_name', 'ip_transform', 'device_num_transform', 'op_days', 'ts', 'ts1', 'ts2','ts3']

remove_col = [x for x in df.columns if x in remove_col]
df = df.drop(columns=remove_col)
df.head()

,id,department,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url,is_risk,url_sit,url_page,op_diff_second1,op_times_groups,device_num_transform_op_times_groups_cumsum,device_num_transform_op_times_groups_department_cumunique,device_num_transform_op_times_groups_ip_transform_cumunique,device_num_transform_op_times_groups_browser_version_cumunique,device_num_transform_op_times_groups_browser_cumunique,device_num_transform_op_times_groups_os_type_cumunique,device_num_transform_op_times_groups_os_version_cumunique,device_num_transform_op_times_groups_ip_type_cumunique,device_num_transform_op_times_groups_http_status_code_cumunique,device_num_transform_op_times_groups_op_city_cumunique,device_num_transform_op_times_groups_log_system_transform_cumunique,device_num_transform_op_times_groups_url_page_cumunique,device_num_transform_op_times_groups_url_sit_cumunique,device_num_transform_op_days_cumsum,device_num_transform_op_days_department_cumunique,device_num_transform_op_days_ip_transform_cumunique,device_num_transform_op_days_browser_version_cumunique,device_num_transform_op_days_browser_cumunique,device_num_transform_op_days_os_type_cumunique,device_num_transform_op_days_os_version_cumunique,device_num_transform_op_days_ip_type_cumunique,device_num_transform_op_days_http_status_code_cumunique,device_num_transform_op_days_op_city_cumunique,device_num_transform_op_days_log_system_transform_cumunique,device_num_transform_op_days_url_page_cumunique,device_num_transform_op_days_url_sit_cumunique,device_num_transform_op_month_cumsum,device_num_transform_op_month_department_cumunique,device_num_transform_op_month_ip_transform_cumunique,device_num_transform_op_month_browser_version_cumunique,device_num_transform_op_month_browser_cumunique,device_num_transform_op_month_os_type_cumunique,device_num_transform_op_month_os_version_cumunique,device_num_transform_op_month_ip_type_cumunique,device_num_transform_op_month_http_status_code_cumunique,device_num_transform_op_month_op_city_cumunique,device_num_transform_op_month_log_system_transform_cumunique,device_num_transform_op_month_url_page_cumunique,device_num_transform_op_month_url_sit_cumunique,http_status_code_op_times_groups_cumsum,ip_transform_op_times_groups_http_status_code_error_code_cumsum,ip_transform_op_times_groups_http_status_code_error_code_cumunique,user_name_op_times_groups_http_status_code_error_code_cumsum,user_name_op_times_groups_http_status_code_error_code_cumunique,device_num_transform_op_times_groups_http_status_code_error_code_cumsum,device_num_transform_op_times_groups_http_status_code_error_code_cumunique,http_status_code_op_days_cumsum,ip_transform_op_days_http_status_code_error_code_cumsum,ip_transform_op_days_http_status_code_error_code_cumunique,user_name_op_days_http_status_code_error_code_cumsum,user_name_op_days_http_status_code_error_code_cumunique,device_num_transform_op_days_http_status_code_error_code_cumsum,device_num_transform_op_days_http_status_code_error_code_cumunique,http_status_code_op_month_cumsum,ip_transform_op_month_http_status_code_error_code_cumsum,ip_transform_op_month_http_status_code_error_code_cumunique,user_name_op_month_http_status_code_error_code_cumsum,user_name_op_month_http_status_code_error_code_cumunique,device_num_transform_op_month_http_status_code_error_code_cumsum,device_num_transform_op_month_http_status_code_error_code_cumunique,hour,hour_sin,hour_cos,dayofweek,dayofweek_sin,dayofweek_cos,day,day_sin,day_cos,ip_transform_op_times_groups_github_cumsum,ip_transform_op_times_groups_github_200_cumsum,ip_transform_op_times_groups_github_400_cumsum,ip_transform_op_times_groups_github_500_cumsum,ip_transform_op_times_groups_github_502_cumsum,ip_transform_op_times_groups_github_404_cumsum,ip_transform_op_times_groups__cumsum,ip_transform_op_times_groups__200_cumsum,ip_transform_op_times_groups__400_cumsum,ip_transform_op_times_groups__500_cumsum,ip_transform_op_times_groups__502_cumsum,

### 评分卡特征筛选

In [15]:
df_row_train = df[df[y_label].notna()].reset_index(drop=True)
df_row_val = df[df[y_label].isna()].reset_index(drop=True)

df_train, df_test, convert_cols = sp.transform_data_detail(df_row_train, df_row_val, y_label, excel_path=path_output_report)
df_train.head()

sheet05.可能为数值类型的object类型数据统计在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖
sheet06.数据预处理在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖


,id,department,browser_version,browser,os_version,op_city,log_system_transform,url,is_risk,url_sit,url_page,op_diff_second1,op_times_groups,device_num_transform_op_times_groups_cumsum,device_num_transform_op_times_groups_url_page_cumunique,device_num_transform_op_days_cumsum,device_num_transform_op_days_log_system_transform_cumunique,device_num_transform_op_days_url_page_cumunique,device_num_transform_op_days_url_sit_cumunique,device_num_transform_op_month_cumsum,device_num_transform_op_month_ip_transform_cumunique,device_num_transform_op_month_http_status_code_cumunique,device_num_transform_op_month_op_city_cumunique,device_num_transform_op_month_log_system_transform_cumunique,device_num_transform_op_month_url_page_cumunique,device_num_transform_op_month_url_sit_cumunique,http_status_code_op_times_groups_cumsum,user_name_op_times_groups_http_status_code_error_code_cumsum,user_name_op_times_groups_http_status_code_error_code_cumunique,http_status_code_op_days_cumsum,user_name_op_days_http_status_code_error_code_cumsum,user_name_op_days_http_status_code_error_code_cumunique,http_status_code_op_month_cumsum,ip_transform_op_month_http_status_code_error_code_cumsum,ip_transform_op_month_http_status_code_error_code_cumunique,user_name_op_month_http_status_code_error_code_cumsum,user_name_op_month_http_status_code_error_code_cumunique,device_num_transform_op_month_http_status_code_error_code_cumsum,device_num_transform_op_month_http_status_code_error_code_cumunique,hour,hour_sin,hour_cos,dayofweek,dayofweek_sin,dayofweek_cos,day,day_sin,day_cos,ip_transform_op_times_groups_github_cumsum,ip_transform_op_times_groups_github_200_cumsum,ip_transform_op_times_groups_download_cumsum,ip_transform_op_times_groups_download_200_cumsum,user_name_op_times_groups_github_cumsum,user_name_op_times_groups_github_200_cumsum,user_name_op_times_groups_download_cumsum,user_name_op_times_groups_download_200_cumsum,device_num_transform_op_times_groups_github_cumsum,device_num_transform_op_times_groups_github_200_cumsum,device_num_transform_op_times_groups_download_cumsum,device_num_transform_op_times_groups_download_200_cumsum,ip_transform_op_days_github_cumsum,ip_transform_op_days_github_200_cumsum,ip_transform_op_days_download_cumsum,ip_transform_op_days_download_200_cumsum,user_name_op_days_github_cumsum,user_name_op_days_github_200_cumsum,user_name_op_days_download_cumsum,user_name_op_days_download_200_cumsum,device_num_transform_op_days_github_cumsum,device_num_transform_op_days_github_200_cumsum,device_num_transform_op_days_download_cumsum,device_num_transform_op_days_download_200_cumsum
0,44477,rd,chrome_93,chrome,win10,深圳,nHrKgKdJ1Mzt,xxx.com/github,1.0,xxx.com,github,NaN,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.500000,0.866025,4,-0.433884,-0.900969,7,0.988468,0.151428,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN
1,45489,rd,safari_13,safari,macos_big_sur_11,成都,nHrKgKdJ1Mzt,xxx.com/github,1.0,xxx.com,github,NaN,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.500000,0.866025,4,-0.433884,-0.900969,7,0.988468,0.151428,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN
2,45706,hr,firefox_78,firefox,win7,深圳,a5G25puBl9xj,hr.xxx.com/,1.0,hr.xxx.com,,NaN,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0.707107,0.707107,4,-0.433884,-0.900969,7,0.988468,0.151428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,45901,rd,edge_93,edge,win10,杭州,nHrKgKdJ1Mzt,xxx.com/github,1.0,xxx.com,github,NaN,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0.866025,0.500000,4,-0.433884,-0.900969,7,0.988468,0.151428,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN
4,43827,sales,ie_9,ie,win10,重庆,sW0whYIx8LFM,work.xxx.com/task,1.0,work.xxx.com,task,NaN,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0.965926,0.258819,4,-0.433884,-0.900969,7

In [16]:
train_woe, val_woe, bins_adj = sp.binning_data_detail(train=df_train,
                        test=df_test, y=y_label, excel_path=path_output_report, var_skip=['id'],)

There are blank strings in 1 columns, which are replaced with NaN. 
 (ColumnNames: url_page)
There are 8 variables have only one binning intervals，please check the binning result. 
 (ColumnNames: ip_transform_op_times_groups_github_cumsum, user_name_op_times_groups_github_cumsum, user_name_op_days_http_status_code_error_code_cumsum, user_name_op_times_groups_github_200_cumsum, device_num_transform_op_times_groups_github_cumsum, ip_transform_op_times_groups_github_200_cumsum, device_num_transform_op_times_groups_github_200_cumsum, user_name_op_times_groups_http_status_code_error_code_cumsum)
sheet07.初始分箱结果在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖
特征ip_transform_op_times_groups_github_cumsum，除特殊值分箱外，分箱箱数少于2箱，无法调整, 建议删除
特征user_name_op_times_groups_github_cumsum，除特殊值分箱外，分箱箱数少于2箱，无法调整, 建议删除
特征user_name_op_days_http_status_code_error_code_cumsum，除特殊值分箱外，分箱箱数少于2箱，无法调整, 建议删除
特征user_name_op_times_group

[INFO] converting into woe values ...


There are blank strings in 1 columns, which are replaced with NaN. 
 (ColumnNames: url_page)


[INFO] converting into woe values ...


There are blank strings in 1 columns, which are replaced with NaN. 
 (ColumnNames: url_page)
sheet09.调整后分箱结果在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖


In [17]:
res_data, res_val, bins_selection = sp.select_feat_detail(train_woe, val_woe,df_test, y_label, bins_adj,exclude=['id'],
                                                iv_max_threshold=1.5,
                                                remove_negitive_coef=False,
                                                corr_threshold=0.7,
                                                #chi_threshold=0.00001,
                                                excel_path=path_output_report)

变量dayofweek_sin 的分箱结果未通过卡方检验
变量dayofweek 的分箱结果未通过卡方检验
变量device_num_transform_op_month_log_system_transform_cumunique 的分箱结果未通过卡方检验
变量op_city 的分箱结果未通过卡方检验
变量url_sit 的分箱结果未通过卡方检验


变量device_num_transform_op_times_groups_download_cumsum_woe被加入
变量http_status_code_op_days_cumsum_woe被加入
变量hour_cos_woe被加入
变量op_diff_second1_woe被加入
变量ip_transform_op_days_github_200_cumsum_woe被加入
变量hour_woe被加入
变量device_num_transform_op_days_log_system_transform_cumunique_woe被加入
变量url_woe被加入
变量http_status_code_op_times_groups_cumsum_woe被加入
变量log_system_transform_woe被加入
变量device_num_transform_op_month_http_status_code_error_code_cumunique_woe被加入
变量device_num_transform_op_times_groups_download_200_cumsum_woe被加入
变量device_num_transform_op_days_cumsum_woe被加入
变量device_num_transform_op_month_url_sit_cumunique_woe被删除


逐步回归结果


,指标1,取值1,指标2,取值2
0,Dep. Variable:,y,No. Observations:,47660.0000
1,Model:,LogisticRegression,Df Residuals:,47646.0000
2,Method:,MLE,Df Model:,13.0000
3,Date:,Thu; 20 Oct 2022,Pseudo R-squ.:,0.4038
4,Time:,09:07:29,Log-Likelihood:,-12562.0000
5,converged:,True,LL-Null:,-21071.0000
6,Covariance Type:,nonrobust,LLR p-value:,0.0000


,feat,coef,std err,z,P>|z|,[0.025,0.975]
0,intercept ...,-1.7064,0.018,-94.076,0.000,-1.742,-1.671
1,log_system_transform_woe ...,-0.4709,0.073,-6.462,0.000,-0.614,-0.328
2,device_num_transform_op_days_log_system_transf...,0.7546,0.091,8.274,0.000,0.576,0.933
3,http_status_code_op_days_cumsum_woe ...,1.4299,0.029,49.266,0.000,1.373,1.487
4,url_woe ...,0.7303,0.057,12.708,0.000,0.618,0.843
5,http_status_code_op_times_groups_cumsum_woe ...,0.1779,0.061,2.893,0.004,0.057,0.298
6,device_num_transform_op_month_http_status_code...,0.2104,0.053,3.986,0.000,0.107,0.314
7,hour_cos_woe ...,1.2317,0.023,53.997,0.000,1.187,1.276
8,hour_woe ...,-0.9363,0.060,-15.575,0.000,-1.054,-0.818
9,device_num_transform_op_days_cumsum_woe ...,-0.0303,0.053,-0.567,0.571,-0.135,0.074


sheet10.特征筛选在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖
sheet11.特征表现在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖


### null importance特征筛选

In [18]:
feats, categorical_feats = get_null_importance(df_train.drop(columns=[y_label,'id']),
                                               df_train[y_label], 
                                               thresholds=15)

In [19]:
# iv_list, iv_drop_var, iv_data = sp.feature_selection_iv(train_woe, bins_adj, 
#                                     y_label, max_threshold=10.0, exclude=['id'])
# print(iv_drop_var)

In [20]:
# corr_matrix, corr_drop_var, corr_data = sp.feature_selection_corr(df_train[], y_label, exclude=['id'])
# print(corr_drop_var)

### 合并特征

In [21]:

object_feats = ['ip_transform', 'user_name', 'device_num_transform']

df_train = pd.concat([df_train_info[object_feats], df_train[feats], res_data], axis=1)
df_test = pd.concat([df_test[feats], res_val], axis=1)

## modeling

In [22]:

feats = df_train.columns.drop(['id', y_label]+object_feats)
feats = feats.drop(categorical_feats)
#feats 

In [23]:
categorical_feats

['department',
 'os_version',
 'log_system_transform',
 'url',
 'url_sit',
 'url_page']

In [43]:
feats = feats_importance.sort_values('importance', ascending=False)[:20]['name'].values

In [44]:

import time
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold

In [45]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

In [46]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []
for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    # kf = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label],
                           # categorical_feature=categorical_feats
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label],
                          #categorical_feature=categorical_feats
                         )
        model = lgb.train(params, train, valid_sets=[val], 
                          num_boost_round=20000, early_stopping_rounds=100)

        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

df_train['oof'] = oof
display(np.mean(score), np.std(score))

score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values('id').reset_index(drop=True)

sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_jupyter,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
[1]	valid_0's auc: 0.932366
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.933266
[3]	valid_0's auc: 0.933804
[4]	valid_0's auc: 0.936443
[5]	valid_0's auc: 0.936836
[6]	valid_0's auc: 0.937006
[7]	valid_0's auc: 0.937553
[8]	valid_0's auc: 0.938441
[9]	valid_0's auc: 0.938297
[10]	valid_0's auc: 0.938051
[11]	valid_0's auc: 0.938463
[12]	valid_0's auc: 0.938414
[13]	valid_0's auc: 0.938558
[14]	valid_0's auc: 0.938602
[15]	valid_0's auc: 0.938592
[16]	valid_0's auc: 0.937638
[17]	valid_0's auc: 0.937512
[18]	valid_0's auc: 0.937756
[19]	valid_0's auc: 0.938175
[20]	valid_0's auc: 0.938303
[21]	valid_0's auc: 0.938233
[22]	valid_0's auc: 0.938148
[23]	valid_0's auc: 0.938269
[24]	valid_0's auc: 0.938474
[25]	valid_0's auc: 0.938481
[26]	valid_0's auc: 0.938624
[27]	valid_0's auc: 0.938507
[28]	valid_0's auc: 0.938553
[29]	valid_0's auc: 0.938482
[30]	valid_0's auc: 0.938552
[31]	valid_0's auc: 0.938446
[32]	valid_0's auc: 0.93826
[33]	v

----------- 2
[1]	valid_0's auc: 0.924972
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.925383
[3]	valid_0's auc: 0.932113
[4]	valid_0's auc: 0.936009
[5]	valid_0's auc: 0.937856
[6]	valid_0's auc: 0.938577
[7]	valid_0's auc: 0.938798
[8]	valid_0's auc: 0.938336
[9]	valid_0's auc: 0.939184
[10]	valid_0's auc: 0.938897
[11]	valid_0's auc: 0.939067
[12]	valid_0's auc: 0.938523
[13]	valid_0's auc: 0.939286
[14]	valid_0's auc: 0.939516
[15]	valid_0's auc: 0.939626
[16]	valid_0's auc: 0.939893
[17]	valid_0's auc: 0.940246
[18]	valid_0's auc: 0.940181
[19]	valid_0's auc: 0.940438
[20]	valid_0's auc: 0.940602
[21]	valid_0's auc: 0.940097
[22]	valid_0's auc: 0.939769
[23]	valid_0's auc: 0.939916
[24]	valid_0's auc: 0.939991
[25]	valid_0's auc: 0.940035
[26]	valid_0's auc: 0.940057
[27]	valid_0's auc: 0.939866
[28]	valid_0's auc: 0.93998
[29]	valid_0's auc: 0.940064
[30]	valid_0's auc: 0.939822
[31]	valid_0's auc: 0.940052
[32]	valid_0's auc: 0.940088
[33]	v

[138]	valid_0's auc: 0.939379
[139]	valid_0's auc: 0.939156
[140]	valid_0's auc: 0.938947
[141]	valid_0's auc: 0.939034
[142]	valid_0's auc: 0.938828
[143]	valid_0's auc: 0.938805
[144]	valid_0's auc: 0.938774
[145]	valid_0's auc: 0.938952
[146]	valid_0's auc: 0.939085
[147]	valid_0's auc: 0.938963
[148]	valid_0's auc: 0.93877
[149]	valid_0's auc: 0.938785
[150]	valid_0's auc: 0.939134
[151]	valid_0's auc: 0.939064
[152]	valid_0's auc: 0.939068
Early stopping, best iteration is:
[52]	valid_0's auc: 0.940407
----------- 4
[1]	valid_0's auc: 0.928535
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.929862
[3]	valid_0's auc: 0.929485
[4]	valid_0's auc: 0.92783
[5]	valid_0's auc: 0.928671
[6]	valid_0's auc: 0.929073
[7]	valid_0's auc: 0.927747
[8]	valid_0's auc: 0.927829
[9]	valid_0's auc: 0.928563
[10]	valid_0's auc: 0.928482
[11]	valid_0's auc: 0.931162
[12]	valid_0's auc: 0.930963
[13]	valid_0's auc: 0.930962
[14]	valid_0's auc: 0.930811
[15]	valid_0's 

,name,importance
1,hour_cos,47942.559878
0,ip_transform_op_times_groups_download_cumsum,32231.130672
2,op_diff_second1,18200.497887
3,http_status_code_op_days_cumsum_woe,14749.170591
5,device_num_transform_op_month_log_system_trans...,14630.183172
11,user_name_op_times_groups_download_cumsum,9315.343606
4,hour,7834.401303
6,ip_transform_op_month_http_status_code_error_c...,7279.251056
8,dayofweek,6160.127141
19,device_num_transform_op_times_groups_download_...,5650.982442


0.9392155881470586

0.0016597723950407765

In [47]:
feats_importance.sort_values('importance', ascending=False)[:50]

,name,importance
1,hour_cos,47942.559878
0,ip_transform_op_times_groups_download_cumsum,32231.130672
2,op_diff_second1,18200.497887
3,http_status_code_op_days_cumsum_woe,14749.170591
5,device_num_transform_op_month_log_system_trans...,14630.183172
11,user_name_op_times_groups_download_cumsum,9315.343606
4,hour,7834.401303
6,ip_transform_op_month_http_status_code_error_c...,7279.251056
8,dayofweek,6160.127141
19,device_num_transform_op_times_groups_download_...,5650.982442


In [48]:
feats_importance.sort_values('importance', ascending=False)[:50].name.values

array(['hour_cos', 'ip_transform_op_times_groups_download_cumsum',
       'op_diff_second1', 'http_status_code_op_days_cumsum_woe',
       'device_num_transform_op_month_log_system_transform_cumunique',
       'user_name_op_times_groups_download_cumsum', 'hour',
       'ip_transform_op_month_http_status_code_error_code_cumsum',
       'dayofweek',
       'device_num_transform_op_times_groups_download_cumsum',
       'device_num_transform_op_month_http_status_code_cumunique',
       'device_num_transform_op_month_ip_transform_cumunique',
       'user_name_op_days_github_cumsum', 'day_cos',
       'ip_transform_op_days_github_cumsum', 'op_diff_second1_woe',
       'device_num_transform_op_month_op_city_cumunique', 'day',
       'hour_cos_woe', 'device_num_transform_op_days_cumsum'],
      dtype=object)

In [49]:
df_train_info['oof']=df_train['oof']

In [50]:

# object_feats = ['ip_transform', 'user_name', 'device_num_transform']

# pd.concat([df_train_info, df_train[['oof','is_risk']]], axis=1).to_csv('caluate.csv', index=False)

In [51]:
df_train_info[abs(df_train_info['is_risk']-df_train_info['oof'])>0.7]['device_num_transform'].value_counts()

HL3vrsyu1H3Z    88
2EmjEhrepKLJ    83
Rfv57YyO3vny    81
0dV6LzVsv7pW    81
pLxAO9Wq3F1j    80
                ..
9nRsk1CCOdLt     1
3uBfpGYbfD3Q     1
7D0GPTvDM4Fn     1
uQRygwTo4gLz     1
PQZXB2FgV30B     1
Name: device_num_transform, Length: 722, dtype: int64

In [42]:
df_train_info[df_train_info['device_num_transform']=='Rfv57YyO3vny']

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk,url_sit,url_page,oof
23670,23670,liulin3167,rd,u9diCFdYZ,Rfv57YyO3vny,safari_13,safari,macos,macos_big_sur_11,2022-01-07 15:25:29,内网,200,深圳,sW0whYIx8LFM,work.xxx.com/task,2022-01,0,work.xxx.com,task,0.034124
13247,13247,liulin3167,rd,u9diCFdYZ,Rfv57YyO3vny,safari_13,safari,macos,macos_big_sur_11,2022-01-07 17:11:16,内网,200,深圳,nHrKgKdJ1Mzt,xxx.com/github,2022-01,0,xxx.com,github,0.015191
3746,3746,liulin3167,rd,u9diCFdYZ,Rfv57YyO3vny,safari_13,safari,macos,macos_big_sur_11,2022-01-07 18:47:44,内网,200,深圳,nHrKgKdJ1Mzt,xxx.com/github,2022-01,0,xxx.com,github,0.032622
11554,11554,liulin3167,rd,u9diCFdYZ,Rfv57YyO3vny,safari_13,safari,macos,macos_big_sur_11,2022-01-10 11:45:47,内网,200,深圳,dwS3cdn15GK4,wpsdoc.xxx.com/kdocs,2022-01,0,wpsdoc.xxx.com,kdocs,0.038250
36562,36562,liulin3167,rd,u9diCFdYZ,Rfv57YyO3vny,safari_13,safari,macos,macos_big_sur_11,2022-01-10 11:45:48,内网,200,深圳,dwS3cdn15GK4,wpsdoc.xxx.com/download,2022-01,0,wpsdoc.xxx.com,download,0.026112
36563,36563,liulin3167,rd,u9diCFdYZ,Rfv57YyO3vny,safari_13,safari,macos,macos_big_sur_11,2022-01-10 11:45:50,内网,200,深圳,dwS3cdn15GK4,wpsdoc.xxx.com/download,2022-01,0,wpsdoc.xxx.com,download,0.027129
36564,36564,liulin3167,rd,u9diCFdYZ,Rfv57YyO3vny,safari_13,safari,macos,macos_big_sur_11,2022-01-10 11:45:51,内网,200,深圳,dwS3cdn15GK4,wpsdoc.xxx.com/download,2022-01,0,wpsdoc.xxx.com,download,0.023711
36565,36565,liulin3167,rd,u9diCFdYZ,Rfv57YyO3vny,safari_13,safari,macos,macos_big_sur_11,2022-01-10 11:45:54,内网,200,深圳,dwS3cdn15GK4,wpsdoc.xxx.com/download,2022-01,0,wpsdoc.xxx.com,download,0.015579
36566,36566,liulin3167,rd,u9diCFdYZ,Rfv57YyO3vny,safari_13,safari,macos,macos_big_sur_11,2022-01-10 11:45:55,内网,200,深圳,dwS3cdn15GK4,wpsdoc.xxx.com/download,2022-01,0,wpsdoc.xxx.com,download,0.028924
36567,36567,liulin3167,rd,u9diCFdYZ,Rfv57YyO3vny,safari_13,safari,macos,macos_big_sur_11,2022-01-10 11:45:56,内网,404,深圳,dwS3cdn15GK4,wpsdoc.xxx.com/download,2022-01,1,wpsdoc.xxx.com,download,0.028118
